In [64]:
import pandas as pd
import numpy as np
import json
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium

from sklearn.cluster import KMeans
print('Libraries Installed!')

Libraries Installed!


In [65]:
data = pd.read_csv('borough.csv')
data.drop(columns=['Unnamed: 0'], inplace = True)
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
print('Data loaded')

Data loaded


## Mapping the boroughs to their respective lattitudes and longitudes

In [66]:
df = pd.merge(data, geo_data, on='Postal Code')
df.head()

Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

## Boroughs and Neighbourhood frequency

In [67]:
print('There are {} boroughs and {} neighbourhoods in the dataframe.'.\
      format(len(df['Borough'].unique()),len(df['Neighbourhood'].unique())))

There are 10 boroughs and 99 neighbourhoods in the dataframe.


## Creating a map of Toronto with neighbourhoods superimposed on top

In [68]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [69]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

## Loading FourSquare Credentials from json

In [70]:
with open('credentials.json') as f:
    cred = json.load(f)
    
print('Loaded Credentials!')

Loaded Credentials!


In [71]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
             cred['CLIENT_ID'], 
            cred['CLIENT_SECRET'], 
            cred['VERSION'], 
            43.753259,-79.329656, 
            500, 
            100)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601235def0a2d3256341b603'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7577590045, 'lng': -79.32343773980772},
   'sw': {'lat': 43.7487589955, 'lng': -79.33587426019228}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label'

In [108]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    no_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        try:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                 cred['CLIENT_ID'], 
                cred['CLIENT_SECRET'], 
                cred['VERSION'],
                lat,
                lng,
                radius, 
                LIMIT)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        
        except :
            no_list.append(name)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    print('Not enough info for {} places in Toronto'.format(len(no_list)))
    return(nearby_venues)

In [109]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
toronto_venues.head()

Not enough info for 0 places in Toronto


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop

## Exploring Toronto Venues

In [110]:
#setting Neighborhood to be the index
toronto_venues.set_index(['Neighborhood'],inplace=True)

toronto_venues.shape

(2114, 6)

In [111]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                             
Agincourt                                                            4   
Alderwood, Long Branch                                               6   
Bathurst Manor, Wilson Heights, Downsview North                     22   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   23   
...                                                                ...   
Willowdale, Willowdale East                                         34   
Willowdale, Willowdale West                                          5   
Woburn                                                               3   
Woodbine Heights                                                     7   
York Mills West                                                      3   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                                             4   
Alderwood, Long Branch                                                6   
Bathurst Manor, Wilson Heights, Downsview North                      22   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    23   
...                                                                 ...   
Willowdale, Willowdale East                                          34   
Willowdale, Willowdale West                                           5   
Woburn                                                                3   
Woodbine Heights                                                      7   
York Mills West                                                       3   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Agincourt                                            4               4   
Alderwood, Long Branch                               6               6   
Bathurst Manor, Wilson Heights, Downsview North     22              22   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   23              23   
...                                                ...             ...   
Willowdale, Willowdale East                         34              34   
Willowdale, Willowdale West                          5               5   
Woburn                                               3               3   
Woodbine Heights                                     7               7   
York Mills West                                      3               3   

                                                 Venue Longitude  \
Neighborhood                                                       
Agincourt                                                      4   
Alderwood, Long Branch                                         6   
Bathurst Manor, Wilson Heights, Downsview North               22   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             23   
...                                                          ...   
Willowdale, Willowdale East                                   34   
Willowdale, Willowdale West                                    5   
Woburn                                                         3   
Woodbine Heights                                               7   
York Mills West                                                3   

                                                 Venue Category  
Neighborhood                                                     
Agincourt                                                     4  
Alderwood, Long Branch                                        6 

In [112]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 unique categories


## Applying get_dummies to categorize Venue Categories

In [113]:
tv = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")

toronto_venues.reset_index(inplace=True)
tv['Neighborhood']=toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [tv.columns[-1]] + list(tv.columns[:-1])
tv = tv[fixed_columns]
print(tv.shape)
tv.head()

(2114, 273)


Yoga Studio  Accessories Store  Airport  Airport Food Court  \
Neighborhood                                                                    
Parkwoods                   0                  0        0                   0   
Parkwoods                   0                  0        0                   0   
Victoria Village            0                  0        0                   0   
Victoria Village            0                  0        0                   0   
Victoria Village            0                  0        0                   0   

                  Airport Gate  Airport Lounge  Airport Service  \
Neighborhood                                                      
Parkwoods                    0               0                0   
Parkwoods                    0               0                0   
Victoria Village             0               0                0   
Victoria Village             0               0                0   
Victoria Village             0               0                0   

                  Airport Terminal  American Restaurant  Antique Shop  ...  \
Neighborhood                                                           ...   
Parkwoods                        0                    0             0  ...   
Parkwoods                        0                    0             0  ...   
Victoria Village                 0                    0             0  ...   
Victoria Village                 0                    0             0  ...   
Victoria Village                 0                    0             0  ...   

                  Trail  Train Station  Vegetarian / Vegan Restaurant  \
Neighborhood                                                            
Parkwoods             0              0                              0   
Parkwoods             0              0                              0   
Victoria Village      0              0                              0   
Victoria Village      0              0                              0   
Victoria Village      0              0                              0   

                  Video Game Store  Vietnamese Restaurant  Warehouse Store  \
Neighborhood                                                                 
Parkwoods                        0                      0                0   
Parkwoods                        0                      0                0   
Victoria Village                 0                      0                0   
Victoria Village                 0                      0                0   
Victoria Village                 0                      0                0   

                  Wine Bar  Wine Shop  Wings Joint  Women's Store  
Neighborhood                                                       
Parkwoods                0          0            0              0  
Parkwoods                0          0            0              0  
Victoria Village         0          0            0              0  
Victoria Village         0          0            0              0  
Victoria Village         0          0            0              0  

[5 rows x 273 columns]

In [129]:
tv.reset_index(inplace=True)

In [138]:
tv_grouped = tv.groupby('Neighborhood').mean().reset_index()
print(tv_grouped.shape)
tv_grouped.head()

(96, 273)


Neighborhood  Yoga Studio  \
0                                        Agincourt          0.0   
1                           Alderwood, Long Branch          0.0   
2  Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                  Bayview Village          0.0   
4                Bedford Park, Lawrence Manor East          0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.043478   

   ...  Trail  Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0  ...    0.0            0.0                            0.0               0.0   
1  ...    0.0            0.0                            0.0               0.0   
2  ...    0.0            0.0                            0.0               0.0   
3  ...    0.0            0.0                            0.0               0.0   
4  ...    0.0            0.0                            0.0               0.0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  \
0                    0.0              0.0       0.0        0.0          0.0   
1                    0.0              0.0       0.0        0.0          0.0   
2                    0.0              0.0       0.0        0.0          0.0   
3                    0.0              0.0       0.0        0.0          0.0   
4                    0.0              0.0       0.0        0.0          0.0   

   Women's Store  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  

[5 rows x 273 columns]

## Clustering Neighborhoods

In [142]:
# set number of clusters
kclusters = 3

tv_cluster = tv_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tv_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[4:18] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])